In [ ]:
import torch
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torch.utils.data import Subset, DataLoader
import torchvision
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
h_params = {
    "epochs":10,
    "learning_rate":0.0001,
    "batch_size":256,
    "num_of_filter":32,
    "filter_size":[5,5,5],
    "actv_func":"relu",
    "filter_multiplier":1,
    "data_augumentation":False,
    "data_normalization":False,
    "dropout":0,
    "conv_layers":3
}

In [ ]:
# Define the desired size
image_size = 800
desired_size = (image_size, image_size)

# Define transformations to be applied to the images
transform = transforms.Compose([
    transforms.Resize(desired_size),  # Resize images to the desired size
    transforms.ToTensor()        # Convert images to tensors
])

# Load the dataset from the folder
data_dir = "/kaggle/input/nature/inaturalist_12K/train"
dataset = ImageFolder(data_dir, transform=transform)
total_images = len(dataset)

batch_size =h_params["batch_size"]
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:

class CNN(nn.Module):
    def __init__(self, h_params):
        super(CNN, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.conv3 = nn.Conv2d(16, 32, 5)
#          self.conv2 = nn.Conv2d(6, 16, 5)
#         self.conv3 = nn.Conv2d(16, 32, 5)
        f_map_side = self.neurons_in_dense_layer([5,5,5], image_size)
        self.fc1 = nn.Linear(32 *f_map_side*f_map_side , 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        # -> n, 3, 32, 32
#         print("input x", x.shape)
        x = self.pool(F.relu(self.conv1(x))) 
#         print("input c1", x.shape)
        x = self.pool(F.relu(self.conv2(x)))
#         print("input c2", x.shape)
        x = self.pool(F.relu(self.conv3(x))) 
#         print("input c3", x.shape)
        f_map_side = self.neurons_in_dense_layer([5,5,5], image_size)
        x = x.view(-1, 32  *f_map_side*f_map_side)   
#         print("input v1", x.shape)
        x = F.relu(self.fc1(x))       
#         print("input f1", x.shape)
        x = self.fc2(x)      
#         print("input f2", x.shape)
        return x
     def neurons_in_dense_layer(self, filter_sizes, image_size):
        for i in range(3):
            image_size = int((image_size - filter_sizes[i] +1)/2)
        return image_size

In [ ]:



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN()
model = torch.nn.DataParallel(model, device_ids = [0,1]).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=h_params["learning_rate"])



for epoch in range(h_params["epochs"]):
    running_loss = 0.0
    correct = 0
    total = 0;
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        total+=inputs.shape[0]

        optimizer.zero_grad()
#         print("input shape:", inputs.shape)
        # Forward pass
        outputs = model(inputs)
#         print(outputs)
#         print(outputs.shape)
#         print(labels)
#         print(labels.shape)

        loss = loss_fn(outputs, labels)

        values, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        loss.backward()
        optimizer.step()

        if i%10 == 9:
          print("epoch: ", epoch+1, " batch no: ", i+1, " loss: ", loss.item())
    print("accuracy:", correct/total_images)
print('Finished Training')
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)


In [ ]:
 # Set the model to evaluation mode
model.eval()
correct = 0
with torch.no_grad():  # Disable gradient calculation to speed up computations
  for inputs, labels in train_loader:
      inputs, labels = inputs.to(device), labels.to(device)

      # Forward pass
      outputs = model(inputs)

      # Compute the predicted labels
      values, predicted = torch.max(outputs, 1)

      # Update counts of correct and total predictions
      correct += (predicted == labels).sum().item()

# Calculate accuracy
accuracy = correct / total_images

print("accuracy: ", accuracy)